# Loading Data

In this workbook we will go over how you can load the data for our exercises

We will be using the data from the PHUSE Open Data Repository

Follow the instructions in [README.md](../README.md) to get setup

##  Data processing in Python 

There are a couple of key libraries we will use:
* [pandas](https://pandas.pydata.org/) - for processing data
* [matplotlib](https://matplotlib.org/) - for creating visual representations of the data
* [lxml](https://lxml.de) - processing the define.xml (or any other XML)

You will find that in the majority of cases someone will have written a module to do what you want to do; all you need to do is be able to find it, and if necessary validate it.  Python Packages are published into the Python Package Index [PyPI](https://pypi.org) so you can search for a module using keywords, for example:
* [Bayesian Analysis](https://pypi.org/search/?q=bayesian)
* [Linear Regression](https://pypi.org/search/?q=linear+regression)
* [ODM](https://pypi.org/search/?q=cdisc+odm)

You can also create your own package repository or build packages from a git repository; this is a good way for a company to facilitate the building out of standard libraries for internal use or building out a validated Python module repository.

In [ ]:
# import the libraries we are going to use

# Pandas data handling library
import pandas as pd
from pandas import DataFrame
# Typing allows you to be typesafe with Python
from typing import Optional
# URLlib is the built in Library for working with the web
import urllib
# requests is a mode
import requests
# lxml is a library for processing XML documents
from lxml import etree
from lxml.etree import _ElementTree


In [ ]:
# define a prefix for where the files can be found
PREFIX = "https://github.com/phuse-org/phuse-scripts/raw/master/data/sdtm/cdiscpilot01/"

def check_link(url: str) -> bool:
    """
    ensure that the URL exists
    :param url: The target URL we will attempt to load
    """
    # this will attempt to open the URL, and extract the response status code
    # - status codes are a HTTP convention for responding to requests
    # 200 - OK
    # 403 - Not authorized   
    # 404 - Not found   
    status_code = urllib.request.urlopen(url).getcode()
    return status_code == 200

def load_cdiscpilot_dataset(domain_prefix: str) -> Optional[DataFrame]:
    """
    load a CDISC Pilot Dataset from the GitHub site
    :param domain_prefix: The two letter Domain prefix that is used to id the dataset
    """
    # define the target for our read_sas directive
    target = f"{PREFIX}{domain_prefix.lower()}.xpt"
    # make sure that the URL exists first
    if check_link(target):
        # load in the dataset 
        dataset = pd.read_sas(target, encoding="utf-8")
        # dataset = pd.read_sas(target)
        return dataset
    return None


def load_cdiscpilot_define() -> _ElementTree:
    """
    load the define.xml for the CDISC Pilot project
    """
    # define the target for our read_sas directive
    target = f"{PREFIX}define.xml"
    # make sure that the URL exists first
    if check_link(target):
        # load in the file 
        page = requests.get(target)
        tree = etree.fromstring(page.content)
        # dataset = pd.read_sas(target)
        return tree
    return None
  

In [ ]:
# Load in a dataset - DM
dm = load_cdiscpilot_dataset('DM')

In [ ]:
# Take a look at a table
dm.head()

In [ ]:
# Generate a Frequency Table for SEX
pd.crosstab(index=dm["SEX"], columns='count', colnames=["SEX"])

In [ ]:
# a two-way frequency table (Age by Sex)
pd.crosstab(index=dm["AGE"], columns=dm["SEX"])

In [ ]:
# Distribution of ages for gender
pd.crosstab(index=dm["AGE"], columns=dm["SEX"]).plot.bar()

In [ ]:
# Generate age distributions
bins = [50, 55, 60, 65, 70, 75, 80, 85, 90]
labels = ["50-55", "55-60", "60-65", "65-70", "70-75", "75-80", "80-85", "85-90"]
dm["AGEBAND"] = pd.cut(dm['AGE'], bins=bins, labels=labels)

In [ ]:
# Plot the data using bands
pd.crosstab(index=dm["AGEBAND"], columns=dm["SEX"]).plot.bar()

In [ ]:
# Load the VS dataset
vs = load_cdiscpilot_dataset('VS')

In [ ]:
# Details on the VS dataset
vs.shape

print(f"Dataset VS has {vs.shape[0]} records")

In [ ]:
# Get the first ten rows

vs.loc[0:10]

In [ ]:
# Generate a distribution for the values

tests = vs.groupby("VSTESTCD")["VSORRES"].sum()

In [ ]:
print(tests)

In [ ]:
# Weird right?  We need to check the type of the column
vs.dtypes


In [ ]:
# ok, that makes sense - an object is not numeric....
tests = vs.groupby("VSTESTCD")["VSSTRESN"].mean().reset_index()

In [ ]:
print(tests)

In [ ]:
# Lets join the DM dataset

labelled = vs.merge(dm, on="USUBJID")
labelled.head()

In [ ]:
labelled_tests = labelled.groupby(["VSTESTCD","SEX", "AGEBAND"])["VSSTRESN"].mean().reset_index()
print(labelled_tests)

In [ ]:
# now, let's look at the define

# the way we do this is to load the content from the URL, and then pass it off to the XML parsing library
odm = load_cdiscpilot_define()

In [ ]:
# XML documents can be treated as a tree, 
# * root item (root)
# * elements (branches)
# * attributes (leaves)

# In this case we have a root item that is an CDISC Operational Data Model (ODM)
# `tag` is the way of working out what type of element we have
print(odm.tag)

# we can look at the attributes using the .get method
print(odm.get("FileOID"))
print(odm.get("CreationDateTime"))


# Namespaces

XML documents use a schema document to define what elements/attributes are permissible (or required/expected).  It is possible to extend a schema to incorporate extra elements/attributes; these attributes exist alongside the existing elements by having them under different namespaces


In [ ]:
# look at the namespaces
print(odm.nsmap)

In [ ]:
# in this the default namespace is ODM 1.2, with define.xml present in the def namespace

# let's get the MetadataVersion element
nsmap = odm.nsmap
nsmap["ODM"] = odm.nsmap.get(None)
mdv = odm.find(".//ODM:MetaDataVersion", nsmap)

In [ ]:
# let's take a look at the define attributes
define_ns = nsmap.get('def')
print(define_ns)
# get the define version
for attribute in ("DefineVersion", "StandardName", "StandardVersion"):
    # attributes should be prefixed with the namespace
    attr = f"{{{define_ns}}}{attribute}"
    if mdv.get(attr):
        print(f"{attribute} -> {mdv.get(attr)}")


In [ ]:
# Remember the Standard Version here!  We will come back to it

# you can scan over the different child elements using the findall method
for itemdef in mdv.findall("./ODM:ItemDef", namespaces=nsmap):
    if itemdef.find("./ODM:CodeListRef", namespaces=nsmap) is not None:
        codelistref = itemdef.find("./ODM:CodeListRef", namespaces=nsmap)
        print(f"Item {itemdef.get('OID')} has CodeList: {codelistref.get('CodeListOID')}")
    else:
        print(f"Item: {itemdef.get('OID')}")

Loading XML is a very useful technique, this example is a simple load and navigate of the define data structure.  I recommend checking out [odmlib](https://pypi.org/project/odmlib/) which is a library that makes processing and manipulation of CDISC ODM documents much more straight forward (written by the venerable [Sam Hume](https://github.com/swhume))

# Summary
In this set we've gone over some elementary activities dealing accessing/loading data; there were some elementary expeditions into how data can be manipulated/visualised using pandas and some simple navigation of an XML document.  

Next we're going to take a look at how accessing data over the web works.